In [2]:
import torch
from torch.nn import functional as F
from torchvision import datasets, transforms
import helper

Exercise: Implement the validation loop below and print out the total accuracy after the loop. You can largely copy and paste the code from above, but I suggest typing it in because writing it out yourself is essential for building the skill. In general you'll always learn more by typing it rather than copy-pasting. You should be able to get an accuracy above 80%.

In [8]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download = True, train = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)

testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download = True, train = False, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = True)

In [52]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim = 1)
        
        return x

In [53]:
model = Classifier()

images, labels = next(iter(testloader))

ps = torch.exp(model(images))

print(ps.shape)

torch.Size([64, 10])


In [12]:
top_p, top_class = ps.topk(1, dim=1)
# Look at the most likely classes for the first 10 examples
print(top_class[:10,:])

tensor([[3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3]])


In [13]:
equals = top_class == labels.view(*top_class.shape)

In [14]:
accuracy = torch.mean(equals.type(torch.FloatTensor))

In [ ]:
# turn off gradients
with torch.no_grad():
    # validation pass here
    for images, labels in testloader:

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.003)

epochs = 30
steps = 0

train_losses, test_losses = [], []

for e in range(epochs):
    
    train_loss = 0
    test_loss = 0
    
    for images, labels in trainloader:
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
    with torch.no_grad():
        for images, labels in testloader:

            ps = torch.exp(model(images))
            loss = criterion(ps, labels)
            test_loss += loss.item()
            
            _, top_class = ps.topk(1, dim =1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy = torch.mean(equals.type(torch.FloatTensor))
            
            
        print(f'Accuracy: {accuracy.item()*100}%')
        print(f"Training loss: {train_loss/len(trainloader)}")
        print(f"Test loss: {test_loss/len(testloader)}")
        
        train_losses.append(train_loss)
        test_losses.append(test_loss)

Accuracy: 87.5%
Training loss: 0.5126520276133185
Test loss: -0.7697855928900895
Accuracy: 87.5%
Training loss: 0.3946964071312947
Test loss: -0.7918734485936013
Accuracy: 87.5%
Training loss: 0.35817688361191546
Test loss: -0.8037810898890161
Accuracy: 93.75%
Training loss: 0.33536012777323915
Test loss: -0.8182605922601784
Accuracy: 87.5%
Training loss: 0.3185675549767673
Test loss: -0.823196261172082


Exercise: Add dropout to your model and train it on Fashion-MNIST again. See if you can get a lower validation loss or higher accuracy.

In [47]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.fc1  = nn.Linear(784, 256)
        self.fc2  = nn.Linear(256, 128)
        self.fc3  = nn.Linear(128, 64)
        self.fc4  = nn.Linear(64, 10)
        
        self.drop = nn.Dropout(p = 0.2)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = self.drop(F.relu(self.fc1(x)))
        x = self.drop(F.relu(self.fc2(x)))
        x = self.drop(F.relu(self.fc3(x)))
        x = F.log_softmax(self.fc4(x), dim = 1)
        
        return x

In [48]:
model = Classifier()
optimizer = optim.Adam(model.parameters(), lr = 0.03)

In [49]:
model.train()

Classifier(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (drop): Dropout(p=0.2, inplace=False)
)

In [51]:
epochs = 15
for e in range(epochs):
    
    train_loss = 0
    model.eval()
    
    for images, labels in trainloader:

        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
    print(train_loss/len(trainloader))
    
    with torch.no_grad():
        model.eval()
        for images, labels in testloader:

            ps = torch.exp(model(images))
            
            _, top_class = ps.topk(1, dim =1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy = torch.mean(equals.type(torch.FloatTensor))
            
        
        model.train()

1.3225104428811876
1.2997024354777102
1.2573873520787082
1.2802611226593252
1.3479482588737504
1.3877762130328588
1.37574692588371
1.3711415760552705
1.371426169679109
1.3933606673913723
1.4210663258648122
1.468695566852464
1.4367428087095209
1.3044231562599191
1.5057679780764874
